## Wealth Management SQL Agent
> *This notebook should work well with the **`Python 3 (ipykernel)`** kernel in SageMaker Studio on ml.t3.medium instance*

In this set of labs we will explore how to build a SQL Agent using Model Context Protocol (MCP) and Strands agents.
Key concepts include:
- Ingesting data into a local [DuckDB](https://duckdb.org/) database
- Creating an [MCP](https://modelcontextprotocol.io/docs/getting-started/intro) Server to expose the database via an API
- Creating a Wealth Management SQL Agent using [Strands Agents](https://strandsagents.com/latest/)


### Use Case Overview
In this use case we will build a Text-to-SQL agent that will serve two distinct personas:
1. **Client**: A client who wants to ask questions about their portfolio For example:
    - "What is the current value of my portfolio?"
    - "How much have I earned in the last month?"
    - "What are my top performing assets?"
2. **Wealth Manager / Portfolio Analyst**: An analysts that can ask questions across all clients. For example:
    - "What is the average portfolio value across all clients?"
    - "Which clients have significant positions in tech stocks?"
    - "Which clients have a portfolio value above $1 million?"

The same database and MCP server will be used for both personas, and we'll demonstrate some simple authorization mechanism that restricts access and capabilities based on the user's role.


### Notebook 1: Data Ingestion
In this notebook, we will create a DuckDB database and populate it with CSV files containing synthetic customer data. The database will be used to answer SQL queries from the agents.

In [ ]:
import sys
import os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

In [ ]:
required_models = [
    "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
]
validate_model_access(required_models)

In [ ]:
import os
import duckdb
from pathlib import Path
from typing import Optional, Union

The data we'll be ingesting is contained in the `data/` directory, which includes:
- `clients.csv`: Contains client information such as name, email, and phone number.
- `accounts.csv`: Contains account information including account ID, client ID, and account type.
- `transactions.csv`: Contains transaction data including transaction ID, account ID, date, amount, and type (credit/debit).
- `holdings.csv`: Contains asset holdings data including holding ID, account ID, asset type, and quantity.

The detailed schema for each table is below:

![Database schema](./_images/db_schema.png)

In [ ]:
def create_table_from_csv(conn: duckdb.DuckDBPyConnection, csv_path: Union[str, Path], table_name: str = None) -> str:
    """
    Helper function to create a table in DuckDB from a CSV file.
    
    Args:
        conn: DuckDB connection object
        csv_path (str): Path to the CSV file
        table_name (str, optional): Name for the table. If None, uses CSV filename.
    
    Returns:
        str: Name of the created table
    
    Raises:
        FileNotFoundError: If CSV file doesn't exist
        Exception: If table creation fails
    """

    csv_file = Path(csv_path)
    if not csv_file.exists():
        raise FileNotFoundError(f"CSV file {csv_path} does not exist")
    
    if table_name is None:
        table_name = csv_file.stem.lower()
    
    table_name = ''.join(c if c.isalnum() else '_' for c in table_name)
    
    try:
        create_table_query = f"""
        CREATE OR REPLACE TABLE {table_name} AS 
        SELECT * FROM read_csv_auto('{csv_file.absolute()}')
        """
        
        conn.execute(create_table_query)
        
        row_count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]

        print(f"Created table '{table_name}' from '{csv_file.name}' with {row_count} rows")

        return table_name
        
    except Exception as e:
        print(f"Error creating table from {csv_file.name}: {str(e)}")
        raise

DuckDB uses a file-based storage system, so the database will be created in the current working directory. The database file will be named `wealth.db`.

In [ ]:
db_path = "wealth.db"
conn = duckdb.connect(db_path)

In [ ]:
# iterate over all CSV files in the data directory and add them to the database
csv_files = Path("data").glob("*.csv")
for csv_file in csv_files:
    try:
        table_name = create_table_from_csv(conn, csv_file)
        print(f"Table '{table_name}' created successfully.")
    except Exception as e:
        print(f"Failed to create table from {csv_file.name}: {str(e)}")

In [ ]:
# confirm successful creation of tables
conn.execute("SHOW TABLES").fetchdf()

Now that the data is ingested, we can run a few simple queries to verify the data.

In [ ]:
conn.execute("SELECT * FROM wealth.clients limit 5").fetchdf()

In [ ]:
conn.execute("SELECT * FROM wealth.transactions limit 5").fetchdf()

In our MCP design, the portfolio analysts agent will have the ability to run SQL queries against the entire database. We don't want to expose the entire database to the client agent, so we'll create a number of [Prepared Statements](https://en.wikipedia.org/wiki/Prepared_statement) that encapsulate common queries and only expose those to the client agent.

The following queries will be exposed to the client agent:
- Get information about the client
- Get information about the client's accounts
- Get information about a specific account's holdings
- Get a listing of all transactions for a specific account

In [ ]:
# get information about a specific customer
conn.execute("PREPARE query_customer_info AS SELECT * FROM clients WHERE client_id = ?")

# get a list of accounts for a specific customer
conn.execute(
    "PREPARE query_customer_accounts AS SELECT * FROM accounts WHERE client_id = ?"
)

# get a list of holdings for a specific account
conn.execute(
    """PREPARE query_account_holdings AS 
    SELECT h.*, a.account_name, a.account_type, a.currency, a.status FROM holdings h inner join accounts a on h.account_id = a.account_id
    WHERE a.client_id = ? and a.account_id = ?"""
)

# get a list of transactions for a specific account
conn.execute(
    """PREPARE query_account_transactions AS 
    SELECT t.*, a.account_name, a.account_type, a.currency, a.status FROM transactions t inner join accounts a on t.account_id = a.account_id
    WHERE a.client_id = ? and a.account_id = ?
    ORDER BY t.trade_date DESC, t.settlement_date DESC"""
)

In [ ]:
# we can now run a few simple queries to verify the data
sample_client_id = "f2cc681e-5180-43fa-864e-fd899a8a8387"
sample_account_id = "acc_c3275420-2902-4884-92b9-1502cca5e202"

conn.execute(f"EXECUTE query_customer_info('{sample_client_id}')").fetchdf().to_dict(orient='records')[0]

By using prepared statements, we can ensure that the client agent can only access the data it needs and eliminate the risk of SQL injection attacks, while the portfolio analyst agent has full access to the database.

In [ ]:
# close the database connection so it can be used by subsequent notebooks
conn.close()